# SSH

This page shows aspects of working with the ssh connection. We will consider an example where an ssh connection is established from one Docker container to another.

## Set up

Let's create the necessary docker entities:

- `ssh_host` container that starts the ssh server - we would be able to connect to this server using ssh.
- `ssh_client` container to which we will give access to the `ssh_host`.
- `network` to allow communication between containers.

In [2]:
docker network create ssh_test_network
docker run -dit --rm --name ssh_host --network ssh_test_network alpine
docker run -dit --rm --name ssh_client --network ssh_test_network alpine

a5e5d0804330499eb0b02eff204b6d8bbcf445961701f7302da30500fce2cb1d
68864676721ced0b6ae7814c85f871f6014d3822e2c3a91f9c0a8d456f315cab
8aa47ff70d428ca0aab8d0ecb61b2094e1bf4125f720f7bdff29c2cc95cf2810


Install and start the ssh server.

In [3]:
docker exec -d ssh_host sh -c " 
# openssh instalation
apk add -q openssh
# generating default host keys - they are necessary for running ssh server
ssh-keygen -A
# running ssh server
/usr/sbin/sshd -D"

Installing ssh client.

In [4]:
docker exec ssh_client apk add -q openssh-client

**Note** Don't forget to stop the container and the network.

In [1]:
docker stop ssh_client ssh_host
docker network rm ssh_test_network

ssh_client
ssh_host
ssh_test_network


## Generating key

You need to generate an ssh key. It consists of two parts, a public key and a private key. You can generate it wherever you like. But in our example we will consider the most straightforward option - generating it on the client.

You can just run `ssh-keygen` and follow the CLI's instructions. It'll ask for filepath for the key and passphrase. However, due to the limitations of the jupyter notebook we are using for the presentation, we need to specify some options:

- `t rsa`: This option specifies the type of key algorithm to use. In this case, it specifies RSA as the algorithm. RSA is a widely used asymmetric encryption algorithm.
- `N ""` : This option sets the passphrase (or password) for the generated key. In your command, it is set to an empty string, effectively creating a key without a passphrase. It means that anyone with access to the private key file can use it without providing a passphrase.
- `f /root/.ssh/id_rsa`: This option specifies the filename and path where the generated key pair will be stored. In this case, it will be stored in the /root/.ssh/ directory with the filename id_rsa. The id_rsa file will contain the private key, while the id_rsa.pub file (automatically generated) will contain the public key.


In [5]:
docker exec ssh_client ssh-keygen -t rsa -N "" -f /root/.ssh/id_rsa

Generating public/private rsa key pair.
Created directory '/root/.ssh'.
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:+BDIYEqA+idEbKfFYGv0FKv8H9mURzJxipslq+QrLMw root@8aa47ff70d42
The key's randomart image is:
+---[RSA 3072]----+
|+o*oo.  . .      |
|o=+*+o . +       |
|ooo+= + = .      |
|.oo.   O =       |
| oo . * S .      |
|  o+.. * .       |
|o. o+ o o        |
|.Eo  o .         |
| . .. .          |
+----[SHA256]-----+


After all I'll have `id_rsa` and `id_rsa.pub` files in my the sepcified for ssh keys folder.

In [6]:
docker exec ssh_client ls /root/.ssh

id_rsa
id_rsa.pub


This is what the private key looks like.

In [7]:
docker exec ssh_client cat /root/.ssh/id_rsa | head -n 10 | tail -n 5

BTFBeF3HVrJAbSsDMHwrQoEuw7lKQ5j4ZQj0QgG6/sCaLog9wQNhkbBIQ0GKQ33T6MKyb0
IKTeBieqcLuzi31BYal444jyqu3vFD/6e+XSF3+7RZ+y4O4z/rinbe0VgDaXgHw31rvFQv
FvgjLkk5qtBYofewY97tiHZNmKWFYctN95WvDEXTGosm6wsLCCpWNZxtba9ash4ClKLCfp
fc/rn8QL8VoU1/SHFfBaXLDbJCLB+tZB7/GoGzEbZAdN19913GQhcMZoIvBR1gDNGryXrt
Z8+DV+nxByo/KZIXBvUotGLtTNMAOhH1xXzNEmy3AAAFiCrCInIqwiJyAAAAB3NzaC1yc2


**Note** Only a few lines can be printed here because Githab, which I use to save these pages, sends me notifications that I have credentials leaks.

And here is what the public key looks like.

In [8]:
docker exec ssh_client cat /root/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDC1spPOD80aZ7Cs4lfu6tXA0An/byUjsANPxDSRkSQN4WviFNeHBEGybOM3SrZBMqinpQ6q98AGG8htD3hLFQ3oO5iBSKQq6X0jiAxCiDsZdKADiyUmWRdm4m61vQ+OfAdDzOE0TVp8Bn5mOU9iXRGH/sql32EXMuaZvRdG3+SyLEZ7Gnq/mGKhVmL2ZyMxAYFMUF4XcdWskBtKwMwfCtCgS7DuUpDmPhlCPRCAbr+wJouiD3BA2GRsEhDQYpDfdPowrJvQgpN4GJ6pwu7OLfUFhqXjjiPKq7e8UP/p75dIXf7tFn7Lg7jP+uKdt7RWANpeAfDfWu8VC8W+CMuSTmq0Fih97Bj3u2Idk2YpYVhy033la8MRdMaiybrCwsIKlY1nG1tr1qyHgKUosJ+l9z+ufxAvxWhTX9IcV8FpcsNskIsH61kHv8agbMRtkB03X33XcZCFwxmgi8FHWAM0avJeu1nz4NX6fEHKj8pkhcG9Si0Yu1M0wA6EfXFfM0SbLc= root@8aa47ff70d42


## Copying public key

On the server, list all the public keys that have access in the `/root/.ssh/authorised_keys` file. In our case, there would only be one line corresponding to the generated pre-public key.

In [9]:
docker exec ssh_client cat /root/.ssh/id_rsa.pub > authorized_keys
docker exec ssh_host mkdir /root/.ssh
docker cp authorized_keys ssh_host:/root/.ssh/authorized_keys
rm authorized_keys

Successfully copied 3.58kB to ssh_host:/root/.ssh/authorized_keys


**Note** files copied with the `docker ps` command have the wrong permissions and ownership - the following cell fixes this.

In [10]:
docker exec ssh_host chown root:root /root/.ssh/authorized_keys
docker exec ssh_host chmod 644 /root/.ssh/authorized_keys

## Getting acceess

If setup correcd you can get ssh from the client server to the host server using command `ssh <user_name>@<host_name>`. In our example try:

```bash
docker exec -it ssh_client ssh root@ssh_host
```

So here is a rather tricky case - we have access to the `ssh_host` CLI through the `ssh_client` CLI.

The following cell shows that it works: we create files in `ssh_host`using `ssh_client`.

In [11]:
docker exec ssh_client\
    ssh -o StrictHostKeyChecking=no root@ssh_host touch file{1..5}
docker exec ssh_host ls /root

file1
file2
file3
file4
file5


**Note** The `-o StrictHostKeyChecking=no` option is used here to prevent the client from asking if it needs to add host to the list of known hosts.